In [47]:
#%%writefile send.py
#!/usr/bin/env python
import pika
import sys

command =''

if len(sys.argv)<2:
    print("Please Specify the command (as a single string) to run as argument to this script.")
    exit(1)
else:
    command = ' '.join(str(x) for x in sys.argv[1:])

connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.queue_declare(queue='hello')

channel.basic_publish(exchange='',
                      routing_key='hello',
                      body=command)
print(" [x] Sent 'Hello World!'")
connection.close()

Overwriting send.py


In [79]:
#%%writefile receive.py
#!/usr/bin/env python
import pika
import shlex
import subprocess

def run_command(command):
    try:
        process = subprocess.Popen(shlex.split(command), stdout=subprocess.PIPE)
    except FileNotFoundError as e:
        print(e)
        return (-1)
        
    while True:
        output = process.stdout.readline().decode()
        if output == '' and process.poll() is not None:
            break
        if output:
            print (output.strip())
    rc = process.poll()
    return rc

connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.queue_declare(queue='hello')

def callback(ch, method, properties, body):
    print(" [x] Command to Run %r" % body)
    run_command(body.decode("utf-8") )
    print("Finished...")

channel.basic_consume(callback,
                      queue='hello',
                      no_ack=True)

print(' [*] Waiting for messages. To exit press CTRL+C')
channel.start_consuming()

Overwriting receive.py
